# Modelling a centrifugal compressor


In [1]:
from OCC.Core.BRepTools import breptools_Read
from OCC.Core.TopoDS import *
from OCC.Core.BRep import BRep_Builder
from OCC.Core.gp import *
from OCC.Core.BRepBuilderAPI import *
from math import pi,degrees,radians
from OCC.Core.BRepTools import breptools_Write
import OCCUtils
import OCCUtils.edge
#import numpy as np
import OCCUtils.face
from OCC.Core.BRepAlgoAPI import *
from OCC.Core.BRepMesh import BRepMesh_IncrementalMesh
from OCC.Core.BRepAdaptor import *
#import HackerCAD
from OCC.Core.GCE2d import *
from OCC.Core.Geom import *
from OCC.Core.BRepPrimAPI import *

from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

![image](./blade_root_strategy.jpg)

In [35]:
# Here we load the root_face. The root face is the face where the root
# each blade is attached. This face was created in FreeCAD and is loaded
# here. It is a revolved b-spline.
file_shape = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(file_shape,"./root_surface.brep",builder)
face_b = OCCUtils.Topo(file_shape).faces().__next__()

In [36]:
renderer = JupyterRenderer()

In [37]:
renderer.DisplayShape(face_b)

In [38]:
renderer

Here, the root surface is inspected.

In [39]:
bas = BRepAdaptor_Surface(face_b)

In [40]:
bas.FirstUParameter(), bas.LastUParameter()

(0.0, 6.28318530717959)

In [41]:
bas.FirstVParameter(), bas.LastVParameter()

(0.0, 1.0)

In [42]:
bas.IsUPeriodic()

True

In [43]:
bas.IsVPeriodic()

False

### Drawing blade root line on the blade root surface

Next I am trying to create an edge on root_face that will be the edge where the compressor blade meets the blade root surface.  In FreeCAD I drew a planar face, and then drew a b-spline on the face.  I will compute the p-curve of the b-spline on the planar face, and then use that p-curve on the the blade root_face to make the blade root edge. As you will see below, I am able to compute the p-curve, but have not been able to make the edge on the blade root face.

In [44]:
# Planar face
ctl_domain_brep_file_contents = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(ctl_domain_brep_file_contents,"./ctl_domain.brep",builder)
face_a = OCCUtils.Topo(ctl_domain_brep_file_contents).faces().__next__()
del ctl_domain_brep_file_contents

In [45]:
renderer.DisplayShape(face_a)

In [46]:
# b-spline on planar face
file_contents = TopoDS_Shape()
builder = BRep_Builder()
assert breptools_Read(file_contents,"./root_curve_on_ctl_domain.brep",builder)
edge_root_curve_3d_on_ctl_domain = OCCUtils.Topo(file_contents).edges().__next__()
del file_contents

In [47]:
bas_ctl_domain = BRepAdaptor_Surface(face_ctl_domain)

In [48]:
bas_ctl_domain.FirstUParameter(),bas_ctl_domain.LastUParameter()

(-200.0, 200.0)

In [49]:
bas_ctl_domain.FirstVParameter(),bas_ctl_domain.LastVParameter()

(-50.0, 50.0)

In [50]:
bas_ctl_domain.Value(0,0)

< gp_Pnt: 200.0, 50.0, 0.0 >

In [51]:
bac = BRepAdaptor_Curve(edge_root_curve_3d_on_ctl_domain,face_a)

In [52]:
cos = bac.CurveOnSurface()

In [53]:
hcurve = cos.GetCurve()

In [54]:
bspline = cos.BSpline()

In [55]:
an_edge = BRepBuilderAPI_MakeEdge(bspline).Edge()

In [56]:
renderer.DisplayShape(an_edge)

In [57]:
a_wire = BRepBuilderAPI_MakeWire(an_edge).Wire()
mf = BRepBuilderAPI_MakeFace(surf, a_wire)

In [58]:
renderer.DisplayShape(mf.Face())

In [21]:
hcurve.__class__

OCC.Core.Adaptor2d.Adaptor2d_HCurve2d

In [22]:
gas = bas.Surface()
gas.__class__

OCC.Core.GeomAdaptor.GeomAdaptor_Surface

In [21]:
hcurve.__class__

OCC.Core.Adaptor2d.Adaptor2d_HCurve2d

In [23]:
surf = bas.Surface().Surface()

In [24]:
surf

<class 'Geom_Surface'>